In [1]:
import pandas as pd
from scipy.stats import chi2_contingency
import glob
from google.cloud import bigquery
from google.oauth2 import service_account
from scipy.stats import shapiro
from scipy.stats import levene
from scipy.stats import bartlett
from scipy.stats import anderson
from scipy.stats import ttest_ind

In [2]:
# 파이썬 구글 빅쿼리 연동 코드
# json 파일

key_path = glob.glob("*.json")[0]
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)

In [3]:
# sql 추출 및 데이터 프레임 변환

def sql_to_dataframe(sql:str) -> pd.DataFrame:
    """
    Args:
        sql (str): sql for extraction

    Returns:
        pd.DataFrame: extract data with sql
    """
    query_job = client.query(sql)
    df = query_job.to_dataframe()
    return df

### 카이제곱 검정

In [4]:
sql = """
SELECT
  user.variant,
  COUNT(CASE WHEN game.user_id IS NOT NULL THEN user.user_id END) AS completed,
  COUNT(CASE WHEN game.user_id IS NULL THEN user.user_id END) AS not_completed,
FROM `ABTest.exp_assignment` user
LEFT JOIN `ABTest.game_actions` game on user.user_id = game.user_id AND game.action = "onboarding complete"
WHERE exp_name = "Onboarding"
GROUP BY user.variant
"""

df = sql_to_dataframe(sql)
df

,variant,completed,not_completed
0,control,36268,13629
1,variant 1,38280,11995


In [5]:
control = [df.loc[0,"completed"], df.loc[0,"not_completed"]]
variant = [df.loc[1,"completed"], df.loc[1,"not_completed"]]
chi2, p_val, d_f, expected = chi2_contingency([control, variant])
print("카이제곱 통계량 : {}".format(chi2))
print("p_value: {}".format(p_val))

카이제곱 통계량 : 156.89437219404937
p_value: 5.397977210897445e-36


### 2표본 t-검정

In [6]:
sql = """
SELECT
    user.variant,
    user.user_id,
    SUM(COALESCE(purchase.amount,0)) AS amount
  FROM `ABTest.exp_assignment` user
  LEFT JOIN `ABTest.game_purchases` purchase on user.user_id = purchase.user_id
  WHERE user.exp_name = "Onboarding"
  GROUP BY user.variant, user.user_id
"""

df = sql_to_dataframe(sql)
df

,variant,user_id,amount
0,control,1000,0.0
1,control,1002,0.0
2,control,1004,0.0
3,control,1006,0.0
4,control,1008,0.0
...,...,...,...
100167,variant 1,101161,0.0
100168,variant 1,101162,0.0
100169,variant 1,101163,0.0
100170,variant 1,101169,0.0


In [7]:
control = df.loc[df["variant"] == "control","amount"]
variant = df.loc[df["variant"] == "variant 1","amount"]
ttest_ind(control, variant,alternative = "two-sided")

Ttest_indResult(statistic=0.7765028422840674, pvalue=0.43745397673126385)

In [8]:
sql = """
SELECT
    user.variant,
    user.user_id,
    SUM(COALESCE(purchase.amount,0)) AS amount
  FROM `ABTest.exp_assignment` user
  LEFT JOIN `ABTest.game_purchases` purchase ON user.user_id = purchase.user_id
  INNER JOIN `ABTest.game_actions` game ON user.user_id = game.user_id AND game.action = "onboarding complete"
  WHERE user.exp_name = "Onboarding"
  GROUP BY user.variant, user.user_id
"""

df = sql_to_dataframe(sql)
df

,variant,user_id,amount
0,control,1000,0.0
1,control,1002,0.0
2,control,1004,0.0
3,control,1006,0.0
4,control,1008,0.0
...,...,...,...
74543,variant 1,101158,0.0
74544,variant 1,101161,0.0
74545,variant 1,101162,0.0
74546,variant 1,101169,0.0


In [9]:
control = df.loc[df["variant"] == "control", "amount"]
variant = df.loc[df["variant"] == "variant 1", "amount"]
ttest_ind(control, variant, alternative = "greater")

Ttest_indResult(statistic=2.230059135584021, pvalue=0.012873241296382431)

### 구매 전환율

In [10]:
sql = """
SELECT
  variant,
  COUNT(DISTINCT user.user_id) AS total_cohorted,
  COUNT(DISTINCT purchase.user_id) AS purchasers,
  COUNT(DISTINCT purchase.user_id) * 1.0 / COUNT(DISTINCT user.user_id) AS pct_purchased
FROM `ABTest.exp_assignment` user
LEFT JOIN `ABTest.game_purchases` purchase ON user.user_id = purchase.user_id
INNER JOIN `ABTest.game_actions` game ON user.user_id = game.user_id AND game.action = "onboarding complete"
WHERE user.exp_name = "Onboarding"
GROUP BY user.variant
"""

df = sql_to_dataframe(sql)
df

,variant,total_cohorted,purchasers,pct_purchased
0,control,36268,4988,0.137532
1,variant 1,38280,4981,0.130120


In [11]:
control = [df.loc[0,"purchasers"], df.loc[0,"total_cohorted"] - df.loc[0,"purchasers"]]
variant = [df.loc[1,"purchasers"], df.loc[1,"total_cohorted"] - df.loc[1,"purchasers"]]
chi2, p_val, d_f, expected = chi2_contingency([control, variant])
print("카이제곱 통계량 : {}".format(chi2))
print("p_value: {}".format(p_val))

카이제곱 통계량 : 8.76705127020291
p_value: 0.0030672092210595144
